# HW 5 Question 2

# Setup

In [1]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

## define model and functions

In [2]:
def model(t_u, w0, w1, b):
    return w0 * t_u**2 + w1 * t_u + b

In [3]:
def loss_function(predicted, actual):
    squared_diffs = (predicted - actual)**2
    return squared_diffs.mean()

In [4]:
rates_to_learn_at = [1/x for x in [10, 100, 1000, 10000, 100000]]
rates_to_learn_at

[0.1, 0.01, 0.001, 0.0001, 1e-05]

## Data import

In [5]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

## Split into train and validation sets

In [6]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices  # <1>

(tensor([ 9,  3,  1,  5,  8,  7,  0, 10,  6]), tensor([4, 2]))

In [7]:
training_t_u = t_u[train_indices]
training_t_c = t_c[train_indices]

validation_t_u = t_u[val_indices]
validation_t_c = t_c[val_indices]

training_t_un = 0.1 * training_t_u
validation_t_un = 0.1 * validation_t_u

## Define the training

In [8]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_function(train_t_p, train_t_c)

        # For the valudation step
        with torch.no_grad(): 
            val_t_p = model(val_t_u, *params)
            val_loss = loss_function(val_t_p, val_t_c)
            assert val_loss.requires_grad == False # <2>
            
        optimizer.zero_grad()
        
        train_loss.backward()
        optimizer.step()
        
        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

# Results

In [9]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_t_u = training_t_un, # <1> 
    val_t_u = validation_t_un, # <1> 
    train_t_c = training_t_c,
    val_t_c = validation_t_c)

params

Epoch 1, Training loss 681.2130, Validation loss 651.4102
Epoch 2, Training loss 402.8719, Validation loss 384.9201
Epoch 3, Training loss 240.9870, Validation loss 228.7854
Epoch 500, Training loss 11.9509, Validation loss 2.6261
Epoch 1000, Training loss 9.2196, Validation loss 2.4695
Epoch 1500, Training loss 7.3807, Validation loss 2.4493
Epoch 2000, Training loss 6.1420, Validation loss 2.5055
Epoch 2500, Training loss 5.3073, Validation loss 2.6000
Epoch 3000, Training loss 4.7442, Validation loss 2.7100
Epoch 3500, Training loss 4.3639, Validation loss 2.8218
Epoch 4000, Training loss 4.1066, Validation loss 2.9277
Epoch 4500, Training loss 3.9320, Validation loss 3.0240
Epoch 5000, Training loss 3.8131, Validation loss 3.1090


tensor([ 0.5763, -1.0316, -0.9586], requires_grad=True)

In [10]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_t_u = training_t_un, # <1> 
    val_t_u = validation_t_un, # <1> 
    train_t_c = training_t_c,
    val_t_c = validation_t_c)

params

Epoch 1, Training loss 681.2130, Validation loss 651.4102
Epoch 2, Training loss 681.0050, Validation loss 651.2089
Epoch 3, Training loss 680.7972, Validation loss 651.0076
Epoch 500, Training loss 583.1072, Validation loss 556.4013
Epoch 1000, Training loss 495.5844, Validation loss 471.6566
Epoch 1500, Training loss 417.7606, Validation loss 396.3195
Epoch 2000, Training loss 348.7388, Validation loss 329.5200
Epoch 2500, Training loss 287.7881, Validation loss 270.5494
Epoch 3000, Training loss 234.3075, Validation loss 218.8250
Epoch 3500, Training loss 187.7944, Validation loss 173.8593
Epoch 4000, Training loss 147.8133, Validation loss 135.2299
Epoch 4500, Training loss 113.9663, Validation loss 102.5504
Epoch 5000, Training loss 85.8696, Validation loss 75.4482


tensor([ 0.5721,  0.5694, -0.4349], requires_grad=True)